In [70]:
from itertools import product
import polars as pl
import pandas as pd

In [ ]:
tissues = ["BAT", "BLOOD", "LIVER", "HEART", "WAT-SC", "SKM-GN", "HYPOTH"]
assays = ["ATAC", "TRNSCRPT", "METHYL"]

products = list(product(tissues, assays))

In [ ]:
genomic_data = pl.read_csv("data/TRAINING_REGULATED_NORM_DATA.csv", null_values=["NA"]).filter(
    pl.col("assay").is_in(assays)
).drop("dataset")
feature_to_gene = pl.read_csv("data/FEATURE_TO_GENE.csv", null_values=["NA"])

In [ ]:
data: pl.DataFrame = genomic_data.join(
    feature_to_gene.select(pl.col("feature_ID", "ensembl_gene")),
    on="feature_ID",
    how="left",
).drop(pl.selectors.by_index(0))

In [ ]:
final_data = data.drop("tissue", "assay", "feature_ID").group_by("feature", "ensembl_gene").mean().sort("ensembl_gene").drop("ensembl_gene")

In [72]:
# Reading phenotype data
pheno = pd.read_csv("phenotype.csv")

# Subsetting phenotype data to include necessary columns
phenoSubset = pheno[["pid",
                     "viallabel",
                     "sex",
                     "vo2.max.test.vo2_max_2",
                     "registration.weight",
                     "terminal.weight.bw",
                     "group"]]

phenoSubset["mass_lost"] = phenoSubset["registration.weight"] - phenoSubset["terminal.weight.bw"]

phenoSubset["norm_mass_lost"] = phenoSubset["mass_lost"] / phenoSubset["registration.weight"]

phenoSubset["norm_mass_lost"] = phenoSubset["mass_lost"] / phenoSubset["registration.weight"]

/var/folders/9n/hn4ksfv50tj5y5b19_rj3wz40000gn/T/ipykernel_11156/3151364854.py:2: DtypeWarning: Columns (37,40,41,42,44,50,66,81,82,104,112,120,128,137,147,156,165,174,183,184,186,193,194,196,201,202,204,209,210,212,217,218,220,226,227,229,236,237,239,244,245,247,252,253,255,260,261,263,269,270,272,279,280,282,287,288,290,295,296,298,303,304,306,312,313,315,322,323,325,330,331,333,338,339,341,346,347,349,355,356,358,365,366,368,373,374,376,381,382,384,389,390,392,398,399,401,403,404,406,411,412,414,419,420,422,428,429,431,437,455,456,459,460,461,463,465,471,473,492) have mixed types. Specify dtype option on import or set low_memory=False.
  pheno = pd.read_csv("phenotype.csv")


KeyError: 'mass_lost'

In [ ]:
phenoSubset.head()

In [ ]:
phenoSubset.to_csv("phenoSubset.csv")